In [ ]:
# LexiBrief - Legal Document Summarizer
This notebook runs the LexiBrief project using Google Colab's GPU. Follow the cells in order to set up and run the project.


In [ ]:
# Check if GPU is available
import torch
print("GPU available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU device:", torch.cuda.get_device_name(0))


In [ ]:
# Upload project files
from google.colab import files
import os
import shutil
import zipfile

print("Please prepare your project files:")
print("1. Zip your entire LexiBrief project folder")
print("2. Make sure to exclude any sensitive files (.env, etc.)")
print("3. Click 'Choose Files' when prompted and select your zip file")

# Upload the zip file
uploaded = files.upload()
zip_file_name = list(uploaded.keys())[0]

# Extract the uploaded zip file
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall('.')

# Clean up the zip file
os.remove(zip_file_name)

# Install dependencies
!pip install -r LexiBrief/requirements.txt

# Change to project directory
%cd LexiBrief

print("\n✅ Project files uploaded and dependencies installed")


In [ ]:
# Set up environment variables
import os
from dotenv import load_dotenv

# Create .env file with your OpenRouter API key
with open('.env', 'w') as f:
    f.write('OPENROUTER_API_KEY=your_api_key_here')  # Replace with your actual API key

# Load environment variables
load_dotenv()


In [ ]:
# Update training configuration for GPU
import yaml

config_updates = {
    'training': {
        'per_device_train_batch_size': 4,  # Increased for GPU
        'per_device_eval_batch_size': 4,   # Increased for GPU
        'gradient_accumulation_steps': 4    # Adjusted for GPU
    },
    'hardware': {
        'mixed_precision': 'fp16',          # Enable mixed precision for GPU
        'device': 'cuda'                    # Set to GPU
    }
}

# Read existing config
with open('configs/training_config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Update config
def update_dict(d, u):
    for k, v in u.items():
        if isinstance(v, dict):
            d[k] = update_dict(d.get(k, {}), v)
        else:
            d[k] = v
    return d

config = update_dict(config, config_updates)

# Save updated config
with open('configs/training_config.yaml', 'w') as f:
    yaml.dump(config, f, default_flow_style=False)


In [ ]:
# Import and run your training script
from train import train  # Adjust this import based on your actual training script name and function

# Run training
train()
